This file is produced by Hengyi Li and Jingbo Wang

2023.3.10

# Fundamentals of machine learning

## Generalization: The goal of machine learning

We can setup our envrinment by importing those libraries 

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import imdb

Before start, we can check our running environment by running those code

In [12]:
print(tf.config.list_physical_devices('GPU'))
print(tf.config.list_physical_devices('CPU'))

[]
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


## Improving generalization

To improve the generalization, we have several ways to do it. 

* Dataset curation

* Feature engineering

* Using early stopping

* Reducing the network's size

  * Reducing the network's size
  * Adding L2 weight regularization to the model
  * Adding dropout to the IMDB model

### Dataset curation

* Make sure you have enough data. Remember that you need a _**dense sampling**_ of the input-cross-output space. More data will yield a better model. Sometimes, problems that seem impossible at first become solvable with a larger dataset.
* Make sure you have enough data. Remember that you need a dense sampling of the input-cross-output space. More data will yield a better model. Sometimes, problems that seem impossible at first become solvable with a larger dataset.
* Clean your data and deal with missing values.
* If you have many features and you aren’t sure which ones are actually useful, do feature selection.


So the num_words restricted the top 10,000 most frequently occurring words in the training data. Rare words will be discarded. And we can do better than that by increasing the num_words to 25000 

**Original Model**

In [13]:
(train_data, train_labels), _ = imdb.load_data(num_words=10000)

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_original = model.fit(train_data, train_labels,
                             epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 1s 20ms/step - loss: 0.5656 - accuracy: 0.7586 - val_loss: 0.4421 - val_accuracy: 0.8590
Epoch 2/20
30/30 [==============================] - 0s 11ms/step - loss: 0.3646 - accuracy: 0.8884 - val_loss: 0.3380 - val_accuracy: 0.8854
Epoch 3/20
30/30 [==============================] - 0s 11ms/step - loss: 0.2689 - accuracy: 0.9153 - val_loss: 0.3165 - val_accuracy: 0.8726
Epoch 4/20
30/30 [==============================] - 0s 11ms/step - loss: 0.2161 - accuracy: 0.9311 - val_loss: 0.2798 - val_accuracy: 0.8917
Epoch 5/20
30/30 [==============================] - 0s 11ms/step - loss: 0.1801 - accuracy: 0.9419 - val_loss: 0.2911 - val_accuracy: 0.8810
Epoch 6/20
30/30 [==============================] - 0s 12ms/step - loss: 0.1547 - accuracy: 0.9494 - val_loss: 0.3013 - val_accuracy: 0.8801
Epoch 7/20
30/30 [==============================] - 0s 10ms/step - loss: 0.1315 - accuracy: 0.9601 - val_loss: 0.2868 - val_accuracy: 0.8870
Epoch 8/20
30

**Practice Model**

Here, we change the num_words to 25000 

In [14]:
(train_data, train_labels), _ = imdb.load_data(num_words=25000)

def vectorize_sequences(sequences, dimension=25000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_original = model.fit(train_data, train_labels,
                             epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 2s 43ms/step - loss: 0.5127 - accuracy: 0.7799 - val_loss: 0.3726 - val_accuracy: 0.8796
Epoch 2/20
30/30 [==============================] - 1s 23ms/step - loss: 0.2925 - accuracy: 0.9111 - val_loss: 0.2962 - val_accuracy: 0.8922
Epoch 3/20
30/30 [==============================] - 1s 23ms/step - loss: 0.2099 - accuracy: 0.9363 - val_loss: 0.3140 - val_accuracy: 0.8706
Epoch 4/20
30/30 [==============================] - 1s 23ms/step - loss: 0.1540 - accuracy: 0.9563 - val_loss: 0.2772 - val_accuracy: 0.8901
Epoch 5/20
30/30 [==============================] - 1s 24ms/step - loss: 0.1210 - accuracy: 0.9655 - val_loss: 0.2875 - val_accuracy: 0.8849
Epoch 6/20
30/30 [==============================] - 1s 22ms/step - loss: 0.0946 - accuracy: 0.9765 - val_loss: 0.3354 - val_accuracy: 0.8749
Epoch 7/20
30/30 [==============================] - 1s 22ms/step - loss: 0.0776 - accuracy: 0.9826 - val_loss: 0.2995 - val_accuracy: 0.8890
Epoch 8/20
30

As it shown above, after change the num_words and the batch size, make a bigger dataset and more training on each epoch, the accuracy gets visible improvement

### Feature engineering

**Original Model**

In [15]:
(train_data, train_labels), _ = imdb.load_data(num_words=10000)

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_original = model.fit(train_data, train_labels,
                             epochs=50, batch_size=512, validation_split=0.4)

Epoch 1/50
30/30 [==============================] - 1s 21ms/step - loss: 0.5278 - accuracy: 0.7821 - val_loss: 0.4018 - val_accuracy: 0.8582
Epoch 2/50
30/30 [==============================] - 0s 12ms/step - loss: 0.3294 - accuracy: 0.8903 - val_loss: 0.3144 - val_accuracy: 0.8833
Epoch 3/50
30/30 [==============================] - 0s 11ms/step - loss: 0.2449 - accuracy: 0.9177 - val_loss: 0.2955 - val_accuracy: 0.8821
Epoch 4/50
30/30 [==============================] - 0s 11ms/step - loss: 0.2016 - accuracy: 0.9293 - val_loss: 0.3300 - val_accuracy: 0.8666
Epoch 5/50
30/30 [==============================] - 0s 12ms/step - loss: 0.1659 - accuracy: 0.9445 - val_loss: 0.2801 - val_accuracy: 0.8873
Epoch 6/50
30/30 [==============================] - 0s 11ms/step - loss: 0.1462 - accuracy: 0.9510 - val_loss: 0.2860 - val_accuracy: 0.8865
Epoch 7/50
30/30 [==============================] - 0s 12ms/step - loss: 0.1241 - accuracy: 0.9601 - val_loss: 0.3043 - val_accuracy: 0.8817
Epoch 8/50
30

**Practice Model**

We can enhance by deleting the most common words

In [16]:
(train_data, train_labels), _ = imdb.load_data(num_words=10000)
common_stop_words=["a","an","and","the","in","of","to","that","this","for"]


def delete_common_stop_words(train_data, common_stop_words):
    word_index = imdb.get_word_index()
    for i in range(len(common_stop_words)):
      if word_index[common_stop_words[i]]:
         common_stop_word_index = word_index[common_stop_words[i]]
         for i in range(len(train_data)):
           train_data[i] = list(
               filter((common_stop_word_index).__ne__, train_data[i]))
    return train_data

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = delete_common_stop_words(train_data, common_stop_words)
train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
Practice_Run  = model.fit(train_data, train_labels,
                             epochs=50, batch_size=512, validation_split=0.4)

Epoch 1/50
30/30 [==============================] - 1s 23ms/step - loss: 0.5184 - accuracy: 0.7879 - val_loss: 0.3901 - val_accuracy: 0.8647
Epoch 2/50
30/30 [==============================] - 0s 11ms/step - loss: 0.3147 - accuracy: 0.8981 - val_loss: 0.3163 - val_accuracy: 0.8770
Epoch 3/50
30/30 [==============================] - 0s 13ms/step - loss: 0.2378 - accuracy: 0.9177 - val_loss: 0.2808 - val_accuracy: 0.8909
Epoch 4/50
30/30 [==============================] - 0s 12ms/step - loss: 0.1893 - accuracy: 0.9383 - val_loss: 0.2754 - val_accuracy: 0.8907
Epoch 5/50
30/30 [==============================] - 0s 13ms/step - loss: 0.1569 - accuracy: 0.9487 - val_loss: 0.3335 - val_accuracy: 0.8702
Epoch 6/50
30/30 [==============================] - 0s 12ms/step - loss: 0.1316 - accuracy: 0.9581 - val_loss: 0.3000 - val_accuracy: 0.8839
Epoch 7/50
30/30 [==============================] - 0s 12ms/step - loss: 0.1149 - accuracy: 0.9645 - val_loss: 0.3259 - val_accuracy: 0.8786
Epoch 8/50
30

### Using early stopping

We could using early stopping class in the Keras library to accomplish this step

**Original model**

In [17]:
(train_data, train_labels), _ = imdb.load_data(num_words=10000)

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_original = model.fit(train_data, train_labels,
                             epochs=50, batch_size=512, validation_split=0.4)

Epoch 1/50
30/30 [==============================] - 1s 27ms/step - loss: 0.5245 - accuracy: 0.7747 - val_loss: 0.4068 - val_accuracy: 0.8577
Epoch 2/50
30/30 [==============================] - 0s 12ms/step - loss: 0.3276 - accuracy: 0.8925 - val_loss: 0.3158 - val_accuracy: 0.8827
Epoch 3/50
30/30 [==============================] - 0s 11ms/step - loss: 0.2436 - accuracy: 0.9204 - val_loss: 0.2824 - val_accuracy: 0.8912
Epoch 4/50
30/30 [==============================] - 0s 11ms/step - loss: 0.1952 - accuracy: 0.9363 - val_loss: 0.2759 - val_accuracy: 0.8896
Epoch 5/50
30/30 [==============================] - 0s 13ms/step - loss: 0.1652 - accuracy: 0.9471 - val_loss: 0.2838 - val_accuracy: 0.8865
Epoch 6/50
30/30 [==============================] - 0s 11ms/step - loss: 0.1395 - accuracy: 0.9549 - val_loss: 0.2848 - val_accuracy: 0.8877
Epoch 7/50
30/30 [==============================] - 0s 11ms/step - loss: 0.1194 - accuracy: 0.9631 - val_loss: 0.3062 - val_accuracy: 0.8803
Epoch 8/50
30

So we could apply the early stopping method as shown below

**Practice**

In [18]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor="accuracy",
    min_delta=0,
    patience=5,
    verbose=1,
    mode="auto",
    baseline=0.9,
    restore_best_weights=True,
    start_from_epoch=0,
)
model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])


model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_practice = model.fit(train_data, train_labels,
                             epochs=50, batch_size=512, validation_split=0.4, callbacks=[callback])

Epoch 1/50
30/30 [==============================] - 1s 19ms/step - loss: 0.5136 - accuracy: 0.7876 - val_loss: 0.3814 - val_accuracy: 0.8736
Epoch 2/50
30/30 [==============================] - 0s 12ms/step - loss: 0.3177 - accuracy: 0.8934 - val_loss: 0.3129 - val_accuracy: 0.8802
Epoch 3/50
30/30 [==============================] - 0s 11ms/step - loss: 0.2415 - accuracy: 0.9182 - val_loss: 0.2924 - val_accuracy: 0.8864
Epoch 4/50
30/30 [==============================] - 0s 13ms/step - loss: 0.1947 - accuracy: 0.9345 - val_loss: 0.2749 - val_accuracy: 0.8911
Epoch 5/50
30/30 [==============================] - 0s 11ms/step - loss: 0.1625 - accuracy: 0.9458 - val_loss: 0.2763 - val_accuracy: 0.8892
Epoch 6/50
30/30 [==============================] - 0s 11ms/step - loss: 0.1385 - accuracy: 0.9546 - val_loss: 0.3159 - val_accuracy: 0.8773
Epoch 7/50
30/30 [==============================] - 0s 12ms/step - loss: 0.1194 - accuracy: 0.9627 - val_loss: 0.3018 - val_accuracy: 0.8829
Epoch 8/50
30

In [19]:
len(history_practice.history['loss'])

31

From the experiments above we could saw that the early stopping take action in the epoch 36 with the accuracy of 0.9999. This is our best results so far and it does save a lot of computing resources

### Regularizing your model

#### Reducing the network's size

**Original model**

In [20]:
(train_data, train_labels), _ = imdb.load_data(num_words=10000)

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_original = model.fit(train_data, train_labels,
                             epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 1s 20ms/step - loss: 0.5459 - accuracy: 0.7713 - val_loss: 0.4365 - val_accuracy: 0.8474
Epoch 2/20
30/30 [==============================] - 0s 11ms/step - loss: 0.3489 - accuracy: 0.8904 - val_loss: 0.3260 - val_accuracy: 0.8826
Epoch 3/20
30/30 [==============================] - 0s 11ms/step - loss: 0.2630 - accuracy: 0.9142 - val_loss: 0.2898 - val_accuracy: 0.8885
Epoch 4/20
30/30 [==============================] - 0s 13ms/step - loss: 0.2134 - accuracy: 0.9297 - val_loss: 0.2795 - val_accuracy: 0.8871
Epoch 5/20
30/30 [==============================] - 0s 11ms/step - loss: 0.1780 - accuracy: 0.9425 - val_loss: 0.2798 - val_accuracy: 0.8866
Epoch 6/20
30/30 [==============================] - 0s 11ms/step - loss: 0.1549 - accuracy: 0.9486 - val_loss: 0.2814 - val_accuracy: 0.8888
Epoch 7/20
30/30 [==============================] - 0s 13ms/step - loss: 0.1340 - accuracy: 0.9580 - val_loss: 0.2878 - val_accuracy: 0.8867
Epoch 8/20
30

**Version of the model with lower capacity**

In [21]:
model = keras.Sequential([
    layers.Dense(4, activation="relu"),
    layers.Dense(4, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_smaller_model = model.fit(
    train_data, train_labels,
    epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 1s 18ms/step - loss: 0.6561 - accuracy: 0.5231 - val_loss: 0.6077 - val_accuracy: 0.5410
Epoch 2/20
30/30 [==============================] - 0s 12ms/step - loss: 0.5775 - accuracy: 0.6102 - val_loss: 0.5566 - val_accuracy: 0.7127
Epoch 3/20
30/30 [==============================] - 0s 10ms/step - loss: 0.5259 - accuracy: 0.7959 - val_loss: 0.5271 - val_accuracy: 0.7646
Epoch 4/20
30/30 [==============================] - 0s 10ms/step - loss: 0.4864 - accuracy: 0.8578 - val_loss: 0.5007 - val_accuracy: 0.8031
Epoch 5/20
30/30 [==============================] - 0s 11ms/step - loss: 0.4541 - accuracy: 0.8841 - val_loss: 0.4734 - val_accuracy: 0.8358
Epoch 6/20
30/30 [==============================] - 0s 10ms/step - loss: 0.4202 - accuracy: 0.9043 - val_loss: 0.4434 - val_accuracy: 0.8710
Epoch 7/20
30/30 [==============================] - 0s 10ms/step - loss: 0.3806 - accuracy: 0.9230 - val_loss: 0.4184 - val_accuracy: 0.8521
Epoch 8/20
30

**Version of the model with higher capacity**

In [22]:
model = keras.Sequential([
    layers.Dense(512, activation="relu"),
    layers.Dense(512, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_larger_model = model.fit(
    train_data, train_labels,
    epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 2s 63ms/step - loss: 0.5601 - accuracy: 0.7267 - val_loss: 0.4032 - val_accuracy: 0.8209
Epoch 2/20
30/30 [==============================] - 2s 56ms/step - loss: 0.3260 - accuracy: 0.8662 - val_loss: 0.2855 - val_accuracy: 0.8793
Epoch 3/20
30/30 [==============================] - 2s 56ms/step - loss: 0.2381 - accuracy: 0.9041 - val_loss: 0.2718 - val_accuracy: 0.8866
Epoch 4/20
30/30 [==============================] - 2s 56ms/step - loss: 0.1975 - accuracy: 0.9232 - val_loss: 0.3159 - val_accuracy: 0.8715
Epoch 5/20
30/30 [==============================] - 2s 56ms/step - loss: 0.1372 - accuracy: 0.9487 - val_loss: 0.4029 - val_accuracy: 0.8444
Epoch 6/20
30/30 [==============================] - 2s 55ms/step - loss: 0.1121 - accuracy: 0.9576 - val_loss: 0.3232 - val_accuracy: 0.8833
Epoch 7/20
30/30 [==============================] - 2s 53ms/step - loss: 0.0900 - accuracy: 0.9706 - val_loss: 0.3426 - val_accuracy: 0.8848
Epoch 8/20
30

#### Adding weight regularization

**Adding L2 weight regularization to the model**

In [23]:
model = keras.Sequential([
    layers.Dense(16,
                 kernel_regularizer=regularizers.l2(0.002),
                 activation="relu"),
    layers.Dense(16,
                 kernel_regularizer=regularizers.l2(0.002),
                 activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_l2_reg = model.fit(
    train_data, train_labels,
    epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 1s 18ms/step - loss: 0.6250 - accuracy: 0.7683 - val_loss: 0.4950 - val_accuracy: 0.8570
Epoch 2/20
30/30 [==============================] - 0s 11ms/step - loss: 0.4196 - accuracy: 0.8891 - val_loss: 0.3989 - val_accuracy: 0.8835
Epoch 3/20
30/30 [==============================] - 0s 12ms/step - loss: 0.3414 - accuracy: 0.9123 - val_loss: 0.3680 - val_accuracy: 0.8867
Epoch 4/20
30/30 [==============================] - 0s 11ms/step - loss: 0.3013 - accuracy: 0.9246 - val_loss: 0.3607 - val_accuracy: 0.8835
Epoch 5/20
30/30 [==============================] - 0s 11ms/step - loss: 0.2810 - accuracy: 0.9301 - val_loss: 0.3641 - val_accuracy: 0.8814
Epoch 6/20
30/30 [==============================] - 0s 13ms/step - loss: 0.2642 - accuracy: 0.9351 - val_loss: 0.3853 - val_accuracy: 0.8711
Epoch 7/20
30/30 [==============================] - 0s 11ms/step - loss: 0.2495 - accuracy: 0.9426 - val_loss: 0.3720 - val_accuracy: 0.8795
Epoch 8/20
30

**Different weight regularizers available in Keras**

In [24]:

regularizers.l1(0.001)
regularizers.l1_l2(l1=0.001, l2=0.001)

#### Adding dropout

**Adding dropout to the IMDB model**

In [25]:
model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(16, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_dropout = model.fit(
    train_data, train_labels,
    epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 1s 19ms/step - loss: 0.6535 - accuracy: 0.6089 - val_loss: 0.5649 - val_accuracy: 0.8413
Epoch 2/20
30/30 [==============================] - 0s 12ms/step - loss: 0.5576 - accuracy: 0.7254 - val_loss: 0.4561 - val_accuracy: 0.8695
Epoch 3/20
30/30 [==============================] - 0s 12ms/step - loss: 0.4769 - accuracy: 0.8011 - val_loss: 0.3897 - val_accuracy: 0.8784
Epoch 4/20
30/30 [==============================] - 0s 12ms/step - loss: 0.4091 - accuracy: 0.8400 - val_loss: 0.3342 - val_accuracy: 0.8816
Epoch 5/20
30/30 [==============================] - 0s 11ms/step - loss: 0.3525 - accuracy: 0.8716 - val_loss: 0.3120 - val_accuracy: 0.8844
Epoch 6/20
30/30 [==============================] - 0s 13ms/step - loss: 0.3150 - accuracy: 0.8917 - val_loss: 0.2858 - val_accuracy: 0.8907
Epoch 7/20
30/30 [==============================] - 0s 12ms/step - loss: 0.2679 - accuracy: 0.9129 - val_loss: 0.2780 - val_accuracy: 0.8885
Epoch 8/20
30

## Summary